In [1]:
import numpy as np
from fenics import *
import matplotlib.pyplot as plt
from dolfin import *
import os
import copy
import torch

In [2]:
# Create data storage class
class DataCollector:
    def __init__(self, V):
        self.V = V
        self.data = {
            'D': [],
            'u': [],
            't': []
        }
        
    def collect_snapshot(self, d2, u, t):
        D_array = d2
        # Store in excel
        mesh_coordinates = mesh.coordinates()
        u_values = u.compute_vertex_values(mesh)
        u_array = np.zeros((nx, ny))
        for i, coord in enumerate(mesh_coordinates):
            x_idx = int(coord[0] * nx)
            y_idx = int(coord[1] * ny)
            if x_idx == nx: x_idx -= 1
            if y_idx == ny: y_idx -= 1
            u_array[y_idx, x_idx] = u_values[i]
        # Store arrays directly
        self.data['D'].append(D_array)
        self.data['u'].append(u_array)
        self.data['t'].append(t)
    
    def save_to_file(self, filename):
        # Convert lists to numpy arrays
        D_data = np.array(self.data['D'])
        u_data = np.array(self.data['u'])
        t_data = np.array(self.data['t'])
        
        # Save using numpy's savez
        np.savez(filename, D=D_data, u=u_data, t=t_data)

def GRF2d(Nx, Ny, alpha): ## alpha: Power spectrum decay parameter that controls the roughness/smoothness of the field
    # define wavenumber and coefficients
    kx = np.fft.fftfreq(Nx,1/Nx)  # wavenumbers  generates frequency components for FFT; The scaling 1/Nx and 1/Ny sets the frequency spacing
    ky = np.fft.rfftfreq(Ny,1/Ny) # wavenumbers  It is used for real FFT, giving only non-negative frequencies
    Kx,Ky = np.meshgrid(kx,ky)    # k = (kx,ky) wavevector in 2d: creates 2D arrays of wavenumbers
    K = np.sqrt(Kx**2+Ky**2).T    # wavenumber magnitude |k|

    #
    lmbda = (K+1)**(-alpha)    # decay parameters Creates a power-law decay factor; Higher alpha creates smoother fields; Adding 1 prevents division by zero at K=0
    # Change the mean and standard deviation

    eta = (
        np.random.randn(*K.shape) + 1j*np.random.randn(*K.shape)
    ) # random coefficients Creates complex random numbers; Real and imaginary parts are independently normally distributed; Shape matches the wavevector grid

    # Fourier coefficients of u; Multiplies random coefficients by decay factors; This shapes the power spectrum of the field
    uhat = lmbda * eta

    # transform back via inverse FT; irfft2 performs 2D inverse real FFT; Transforms from frequency domain back to spatial domain; norm='forward' specifies the normalization convention
    u = np.fft.irfft2(uhat, norm='forward')

    return u

def create_diffusion_coefficient(V,d2):
    D = Function(V)
    d = D.vector()
    dof_coordinates = V.tabulate_dof_coordinates()
    x = dof_coordinates[:, 0]
    y = dof_coordinates[:, 1]
    
    grid_x = (x * N).astype(int)
    grid_y = (y * N).astype(int)
    grid_values = d2
    for i in range(len(x)):
        if grid_x[i] >= N:
            grid_x[i] = N-1
        if grid_y[i] >= N:
            grid_y[i] = N-1
        d[i] = grid_values[grid_x[i], grid_y[i]]
    
    return D

In [4]:
nx = ny = 64
mesh = RectangleMesh(Point(0, 0), Point(1, 1), nx, ny)
V = FunctionSpace(mesh, 'P', 1)
data_collector = DataCollector(V)
for i in range(10):
    N = 16
    x = np.linspace(0,1,N)
    X,Y = np.meshgrid(x,x)
    alpha = 3
    d_field = abs(GRF2d(N,N,alpha))
    d2 = copy.deepcopy(d_field)
    nx = ny = 64
    mesh = RectangleMesh(Point(0, 0), Point(1, 1), nx, ny)
    V = FunctionSpace(mesh, 'P', 1)
    
    D = create_diffusion_coefficient(V,d2)
    num_steps = 10
    dt = 0.001
    
    def boundary(x, on_boundary):
        return on_boundary
    
    bc = DirichletBC(V, Constant(0.0), boundary)
    parameter = 1
    x0, y0 = 0.5, 0.5
    magnitude = 1
    
    u_initial = Expression("magnitude*(1-exp(-1/((parameter*parameter*(x[0]-x0)*(x[0]-x0)+" + 
                          "parameter*parameter*(x[1]-y0)*(x[1]-y0)))))",
                          degree=2, magnitude=magnitude, parameter=parameter, x0=x0, y0=y0)
    u_n = interpolate(u_initial, V)
    
    u = TrialFunction(V)
    v = TestFunction(V)
    f = Constant(0.0)
    
    a = u*v*dx + dt*D*inner(grad(u), grad(v))*dx + dt*inner(grad(u), grad(D))*v*dx - dt*inner(grad(D), grad(u))*v*dx
    L = u_n*v*dx + dt*f*v*dx
    
    u = Function(V)
    
    output_dir = "DataSet_test"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    t = 0
    for n in range(num_steps):
        t += dt
        solve(a == L, u, bc)
        data_collector.collect_snapshot(d2, u, t)
        u_n.assign(u)

# Save all collected data after the loops
data_collector.save_to_file(f"{output_dir}/simulation_data.npz")
print(f"Data saved to: {os.path.abspath(output_dir)}/simulation_data.npz")

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p